In [19]:
from pymongo import MongoClient

client = MongoClient('change your connect url adress')
db = client["mydatabase"]
collection = db["kisiler"]

Not: Sorulari Mongo querry kodlari kullanarak cozunuz. 

1. fatih isimli kullanicilari bulunuz

In [20]:
results = collection.find({"name": "fatih"})

for result in results:
    print(result)


2. fatih isimli ve email adresi fatih@gmail.com olani bul?

In [21]:
result = collection.find_one({"name": "fatih", "email": "fatih@gmail.com"})

print(result)

None


3. isminin bas harfi 's' olan kullanicilari bulunuz

In [22]:
result = collection.find({"name": {"$regex": "^f"}})
for person in result:
    print(person)

4. fatih isimli kullanicinin notlarini bulunuz

In [23]:
#Yontem 1

result = collection.find({"name": "fatih"}, {"_id":0,"notlar": 1})

for doc in result:
    print(doc)

#Yontem2
result = collection.find({"name": "fatih"})

for document in result:
    notlar = document.get("notlar")
    print("Notlar:", notlar)


#Yontem3
result = collection.find({"name": "fatih"})

for doc in result:
    print("Notlar:", doc['notlar'])

5. "javascript" projesinde 80 ile 90 arasında bir nota sahip olan kişileri bulunuz

In [24]:
result = collection.find({"project.javascript": {"$elemMatch": {"$gte": 80, "$lte": 90}}},
                                                 {'_id':0,'name':1,"project.javascript":1,'email':1})
for person in result:
    print(person)

6. Ayse'nin 77 olan javascript notunu siliniz.

In [25]:
collection.update_one(
    {"name": "ayse"},
    {"$pull": {"project.javascript": 77}}
)

7. Kullanicilari email adresleri ile birlikte takipci sayilarina gore siralayaniz

In [26]:
results = collection.find({},{'_id':0,'name':1,'follower':1,'email':1}).sort("follower")

for result in results:
    print(result)


8. Takipci sayisi en yuksek 3 kullaniciyi bulunuz

In [27]:
results = collection.find({},{'_id':0,'name':1,'follower':1,'email':1}).sort("follower", -1).limit(3)

for result in results:
    print(result)

9. elifin takipci sayisini 25 arttiriniz?

In [28]:
collection.update_one({"name": "elif"}, {"$inc": {"follower": 25}})
results = collection.find({"name": "elif"},{'_id':0,'follower':1})

for result in results:
    print(result)

10. butun kullanicilarin takipcilerinin ortalamasini bulunuz

In [29]:
pipeline = [
    {"$group": {"_id": None, "average_followers": {"$avg": "$follower"}}}
]

result = collection.aggregate(pipeline)

for doc in result:
    average_followers = doc.get("average_followers")
    print("Takipçi Ortalaması:", average_followers)


11. En dusuk notu olan kullaniciyi bulunuz?

In [30]:
pipeline = [
    {"$project": {"_id": 0, "name": 1, "email": 1, "lowest_score": {"$min": "$notlar"}}},
    {"$sort": {"lowest_score": 1}},
    {"$limit": 1}
]

result = collection.aggregate(pipeline)

for kullanici in result:
    print(kullanici)


12. En yuksek python score u olan ilk 3 kullaniciyi bulunuz?

In [31]:
pipeline = [
    {"$project": {"_id": 0, "name": 1, "email": 1, "python_score": {"$max": "$project.python"}}},
    {"$sort": {"python_score": -1}},
    {"$limit": 3}
]

result = collection.aggregate(pipeline)

for kullanici in result:
    kullanici_adi = kullanici.get("name")
    python_skoru = kullanici.get("python_score")
    print(f"{kullanici_adi} - Python Skoru: {python_skoru}")


13. Kullanicilarin tum proje(python,javascript,c#) skorlarinin ortalamasini bulup bu ortalamada en iyi 3 kisiyi bulunuz?

In [32]:

pipeline = [
    {
        "$project": {
            "_id": 0,
            "name": 1,
            "email": 1,
            "python_avg": {"$avg": "$project.python"},
            "javascript_avg": {"$avg": "$project.javascript"},
            "csharp_avg": {"$avg": "$project.csharp"},
            "total_avg": {
                "$avg": [
                    {"$avg": "$project.python"},
                    {"$avg": "$project.javascript"},
                    {"$avg": "$project.csharp"}
                ]
            }
        }
    },
    {"$sort": {"total_avg": -1}},
    {"$limit": 3}
]

result = collection.aggregate(pipeline)

for kullanici in result:
    print(f"Isim: {kullanici['name']}, Toplam ortalama score: {kullanici['total_avg']}") 


#2. yontem

pipeline = [
    {
        "$project": {
            "_id": 0,
            "name": 1,
            "email": 1,
            "python_avg": {"$avg": "$project.python"},
            "javascript_avg": {"$avg": "$project.javascript"},
            "csharp_avg": {"$avg": "$project.csharp"}
        }
    },
    {
        "$addFields": {
            "total_avg": {
                "$avg": ["$python_avg", "$javascript_avg", "$csharp_avg"]
            }
        }
    },
    {"$sort": {"total_avg": -1}},
    {"$limit": 3}
]

result = collection.aggregate(pipeline)

for kullanici in result:
    print(f"Isim: {kullanici['name']}, Toplam ortalama score: {kullanici['total_avg']}") 


